In [5]:
import numpy as np

from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Embedding
from keras.layers.recurrent import LSTM, GRU

def one_hot(value):
    oh = np.zeros(len(voc), dtype='float32')
    oh[value]=1.
    return oh

def get_seq_and_letter(mat, lens, size, split):
    X_dat = []
    Y_dat = []
    X_test = []
    Y_test = []
    for i in range(int(split * size)):
        X_dat.append(mat[i,0:lens[i]-1])
        Y_dat.append(one_hot(mat[i,lens[i]-1]))
    for i in range(int(split* size), size):
        X_test.append(mat[i,0:lens[i]-1])
        Y_test.append(one_hot(mat[i,lens[i]-1]))
    return X_dat, Y_dat, X_test, Y_test

In [6]:


max_features=len(voc)
emb_size = 32
maxlen = 30 # cut texts after this number of words (among top max_features most common words)                                                                                                                                                 
batch_size = 16

print "Loading data..."
X_train, y_train, X_test, y_test = get_seq_and_letter(mat, lens, 50000, 0.8)
print len(X_train), 'train sequences'
print len(X_test), 'test sequences'

#print X_train[0:10], y_train[0:10]                                                                                                                                                                                                           

print "Pad sequences (samples x time)"
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print 'X_train shape:', X_train.shape
print 'X_test shape:', X_test.shape


Loading data...
40000 train sequences
10000 test sequences
Pad sequences (samples x time)
X_train shape: (40000, 30)
X_test shape: (10000, 30)


In [7]:
print 'Build model...'
model = Sequential()
model.add(Embedding(max_features, emb_size))
model.add(LSTM(emb_size, 128)) # try using a GRU instead, for fun
#model.add(Dropout(0.5))
model.add(Dense(128, len(voc)))
model.add(Activation('softmax'))

# try using different optimizers and different optimizer configs                                                                                                                                                                              
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print "Train..."
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=10, verbose=0)
score = model.evaluate(X_test, y_test, batch_size=batch_size)
print 'Test score:', score

classes = model.predict_classes(X_test, batch_size=batch_size)
acc = np_utils.accuracy(classes, y_test)
print 'Test accuracy:', acc



Build model...
Train...
Test score: 2.68290755932
Test accuracy: 0.0


/usr/local/lib/python2.7/dist-packages/Theano-0.6.0-py2.7.egg/theano/scan_module/scan_perform_ext.py:117: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [8]:
def tostr(u):
    st = ""
    for x in u:
        st+=voc[x]
    return st

def see(X, Y):
    for x,y in zip (X,Y):
        print tostr(x), voc[y]

def seeModel(X, model):
    see(X, model.predict_classes(X))

In [9]:
seeModel(X_test[0:20], model)

*ek though, ev%%%%%%%%%%%%%%%% e
*iments'he%%%%%%%%%%%%%%%%%%%% o
*'quot'negativ%%%%%%%%%%%%%%%%  
*erness.'lt%%%%%%%%%%%%%%%%%%% '
*and james%%%%%%%%%%%%%%%%%%%%  
*he next %%%%%%%%%%%%%%%%%%%%% t
*t the bottom%%%%%%%%%%%%%%%%%  
*, very c%%%%%%%%%%%%%%%%%%%%% o
*sometimes fresh%%%%%%%%%%%%%%  
'warning'lt''em'gt%%%%%%%%%%%% '
' a question %%%%%%%%%%%%%%%%% t
*to the pan if%%%%%%%%%%%%%%%%  
*oked covered wit%%%%%%%%%%%%%  
* your fish, y%%%%%%%%%%%%%%%% e
*using lemon%%%%%%%%%%%%%%%%%%  
*ref''quo%%%%%%%%%%%%%%%%%%%%% t
* the brow%%%%%%%%%%%%%%%%%%%%  
* in your ici%%%%%%%%%%%%%%%%% n
*hay' is litera%%%%%%%%%%%%%%% n
*ore than%%%%%%%%%%%%%%%%%%%%%  
